In [1]:
#Packages to help with analysis
import matplotlib.pyplot as plt
import nltk, re, tweepy 
import numpy as np
import pandas as pd
import pyreadstat
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from collections import Counter
import sklearn
from textblob import TextBlob
from sklearn.manifold import TSNE
from textblob import Word
from spellchecker import SpellChecker
%matplotlib inline



# Let's Take a Look at the Data

In [2]:
pd.set_option('display.max_columns', None)
df, meta = pyreadstat.read_sav('Emotional Well-Being - CASEL COVID-19 Webinar_Anon.sav')
df.head()

,StartDate,EndDate,Progress,Duration__in_seconds_,Finished,RecordedDate,ResponseId,OpenEmotions_1,OpenEmotions_3,OpenEmotions_4,ClosedEmotions_1,ClosedEmotions_2,ClosedEmotions_3,ClosedEmotions_4,ClosedEmotions_5,ClosedEmotions_6,StressCauses_1,StressCauses_12,StressCauses_13,JoyCauses_1,JoyCauses_12,JoyCauses_13,Strats_EffectiveSelf_1,Strats_EffectiveSelf_2,Strats_EffectiveSelf_3,Strats_EffectiveOthe_1,Strats_EffectiveOthe_2,Strats_EffectiveOthe_3,ER_1,ER_2,WellBeingSupport_16,WellBeingSupport_17,WellBeingSupport_18,CASELWebinars_16,CASELWebinars_17,CASELWebinars_18,Age,Race_1,Race_2,Race_3,Race_4,Race_5,Race_6,Race_7,Race_8,Race_8_TEXT,Gender,Gender_3_TEXT,Educator,Role_11,Role_12,Role_1,Role_2,Role_3,Role_4,Role_5,Role_6,Role_7,Role_8,Role_10,Role_10_TEXT,Grade_1,Grade_2,Grade_3,Grade_4,StudentSES_1
0,2020-03-24 16:47:03,2020-03-24 16:52:06,100.0,302.0,1.0,2020-03-24 16:52:06,R_Q64hfjjdCiGnM09,Anxious,Sad,Fearful,6.0,6.0,3.0,5.0,5.0,3.0,uncertainty,News,updates,kids,work,family,stop soical media,running,journaling,,,,NaN,NaN,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
1,2020-03-24 16:48:25,2020-03-24 16:52:45,100.0,259.0,1.0,2020-03-24 16:52:45,R_Anw03Hj4s7jxnfH,anxiety,fear,hope,5.0,6.0,3.0,4.0,3.0,4.0,worry about others,worry about self,loss of normal routine,connecting with others,,,yoga,connecting with others,,listen to them,provide facts,provide resources,3.0,4.0,Less cognitive demands,Show grace and understanding,Listen to what people need,,,,39,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,2.0,,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
2,2020-03-24 16:51:16,2020-03-24 16:52:49,100.0,92.0,1.0,2020-03-24 16:52:49,R_32UYohfSNVrkECi,,,,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,NaN,NaN,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
3,2020-03-24 16:47:38,2020-03-24 16:53:13,100.0,334.0,1.0,2020-03-24 16:53:13,R_24PeWDZ57ldPxMy,Fear,Sadness,Anger,5.0,5.0,4.0,5.0,4.0,3.0,Uncertainty,Change,Finances,Family,Connectedness,Quiet,Breathing,Meditation,Exercise,Calm leadership,Meditation,Optimism,5.0,6.0,Reassurance,Non-education related activities that increase...,"Ways of increasing normalcy, routine, and conn...",,,,36,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,2.0,,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,1.0,5.0
4,2020-03-24 16:46:45,2020-03-24 16:53:26,100.0,400.0,1.0,2020-03-24 16:53:26,R_wMCGldjiVHje0o1,Panic,Terror,Depression,3.0,5.0,4.0,4.0,4.0,5.0,Fear for my health,Fear for my family’s health,Social isolation,My son’s smile,My son’s laughter,Old VMs from a friend,Yoga,EFT,Nature,Outside time,Validating feelings,Hugging,5.0,5.0,Acceptance of all feelings,Tools to use in the moment,Space,Access to physical touch,Self reg strategies,EFT,,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,2.0,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Private SLP,1.0,1.0,NaN,1.0,8.0


In [28]:
most_feelings_current_climate = []
aux = np.append(most_feelings_current_climate, df['JoyCauses_1'])
aux1 = np.append(aux, df['JoyCauses_12'])
aux2strat = np.append(aux1, df['JoyCauses_13'])

all_strats = pd.Series(pd.Series([x.lower() for x in aux2strat]))
ordered_counts_strat = all_strats.value_counts()

len(all_strats)



15483

In [4]:
def ordered_counts(list_):
    aux = pd.Series(list_)
    ordered_counts_strat = aux.value_counts()
    return ordered_counts_strat[0:50]
  

In [5]:
ordered_counts_strat[0:50]

family                          891
                                488
friends                         455
my family                       230
my children                     139
exercise                        128
nature                          126
family time                     107
pets                             99
faith                            96
children                         95
my kids                          93
reading                          90
my husband                       88
my dog                           86
work                             77
music                            75
time with family                 74
my friends                       73
cooking                          70
prayer                           63
sunshine                         58
being outside                    57
kids                             56
god                              54
husband                          53
spending time with family        52
meditation                  

In [6]:

def response_list(open_questions_haystack, needle):
    ##Takes in the open ended data and finds all instances where a word (Neele) is used.
    ##
    ##Returns a list of those items
    response = []
    for i in open_questions_haystack:
        if i.find(needle) >=0:
            response = np.append(response, i)
            

    return(response)




In [65]:

#if i (["my god", "my", "not"]"my god").find("god")
passages = response_list(passages, "with")

passages

array(['Ability to connect with my co-workers', 'work with colleagues',
       'Zooming with co-workers',
       'Zooming with students and colleagues I work with - so much joy!',
       'conversations with friends and co-workers'], dtype='<U63')

In [8]:

def category_addition(open_questions_haystack, list_of_needles):
    #Takes a list of data (a category) and finds all instances eache of the inputed categories are used.
    #
    #returns a list of all responses
    total_words = []
    for i in list_of_needles:
        response = response_list(open_questions_haystack, i)
        #fl = any(word in response for word in total_words)
        total_words = np.append(total_words, response)
    return total_words


In [103]:

def category_addition_correct(open_questions_haystack, category):
    total_words = []
    for i in category:        
        response = response_list(open_questions_haystack, i)
        sub_words = []
        for word in response:
            if word in total_words:
                if word in sub_words:
                    total_words =list(np.append(total_words, word))
                
            else:
                sub_words =list(np.append(sub_words, word))
                total_words =list(np.append(total_words, word))

    return total_words
category_addition_correct(passages, family)

['Ability to connect with my co-workers',
 'Zooming with co-workers',
 'Kindness I have witness amongst co-workers',
 'conversations with friends and co-workers',
 'Zooming with students and colleagues I work with - so much joy!',
 'work with colleagues',
 'work with colleagues']

In [74]:
if not false:

SyntaxError: unexpected EOF while parsing (<ipython-input-74-e6b3f511ec2f>, line 1)

In [47]:
%%time

family = ["son", 'child', 'fami', 'husb', 'wife', 'daughter', 'kid', 'partner', 'spouse', 'parent', 'mom', 
          'babies', 'loved one', 'niece', 'nephew', 'grandm', 'grandf', 'mother', 'father',
          'newborn', 'sibling', 'girl', 'boy', 'sister', 'brother', 'cousin', 'aunt']

l = []
for passage in passages:
    fl = any(word in passage for word in family)
    
    if not fl:
        l.append(passage)

CPU times: user 3.17 ms, sys: 91 µs, total: 3.26 ms
Wall time: 3.24 ms


In [45]:
%%time
[passage for passage in passages if any(word in passage for word in family)]

CPU times: user 2.94 ms, sys: 24 µs, total: 2.97 ms
Wall time: 2.99 ms


['my coworkers, family and friends',
 'conversations with daughter about her work and recognition of her competence and resilience',
 'Finding ways to virtually connect with the children I work with',
 'spending time at home with my kids, even while teleworking it is challenging',
 'My children being excited to still do school work',
 'Connection with loved ones and co-workers',
 'Knowing the resiliency of many parents and children who are accomplishing the school work',
 'My coworkers!  The "can do" and help each other and our students and families out attitude.  It keeps me in awe of the goodness inside so many!',
 'Having my kids happy to show me the work they were doing at home',
 'Unity of family, friends, and coworkers.',
 'Spending time with my partner who works from home.',
 'coworkers donating furniture and items to a homeless family',
 'Staying virtually connected with my family/friend/coworkers',
 "Getting to spend more time with my kids because I haven't had to travel for w

In [104]:
#Create categories
#
family = ["son", 'child', 'fami', 'husb', 'wife', 'daughter', 'kid', 'partner', 'spouse', 'parent', 'mom', 
          'babies', 'loved one', 'niece', 'nephew', 'grandm', 'grandf', 'mother', 'father',
          'newborn', 'sibling', 'girl', 'boy', 'sister', 'brother', 'cousin', 'aunt']
religion = ['faith', 'relig', 'pray', 'god', 'jesus', 'sprit', 'church', 'gospel', 'the lord', 'bible',
            'spiritual']
work = ['colleag', 'job', 'work']
athletics = ['walk', 'exercis', 'run', 'yoga', 'health', 'hiking', 'danc','gym', 'bike', 
             'physically active', 'pilates', 'exer' ]
outdoors = ['outside', 'nature', 'sunshine', 'outdoor', 'fresh air']
friends = ['friend', 'relationship', 'neighbor']
pets = [' pet', 'pet ', 'dog', 'cat ',' cat', 'animal', 'puppy']
reading = ["book", 'reading']
sleep = ['sleep', 'rest']
film = ['tv', 'movie', 'netflix', 'film']
food_and_drink = ['food', 'wine', 'cook', 'bak', 'eating', 'meal', 'grocery']
rest = ['rest', 'sleep']
weather = ['weather', 'spring']
students = ['student']
music = ['music', 'song']
hobbies = ['garden', 'puzzle', 'draw', 'hobb']
virtual = ['virtual', 'zoom', 'facetime', 'face time', 'text', 'phone', 'video', 'technology', 'electronic',
          'telecom', 'social media']
value_time = ['slowing', 'extra time', 'slow down', 'having time', 'free time', 'more time', 'down time']

meditation = ['breath', 'meditat']
compassion =['kindness', 'compassion', 'gratitu', 'kind', 'gratef']

hope = ['hope','optimism' ]


chores_and_projects =  ['cleaning', 'tidy', 'chore', "project", 'task']


humor = ["funny", 'humor', 'comdey', 'comic', 'laugh', 'joke']


connecting = ['connect', 'community', 'communities', 'in touch', 'meet']

helping = ["help", 'generos']

engage_with_work_and_school = ['school','student', 'job', 'client', 'staff','office', 'teach', 'colleag' ]

looking_forward = ["looking forward", ' end', 'will pass', 'shall pass']



##Create a list of categories
##
##
list_of_cat = [family, friends, connecting, virtual, pets]
total_words = []
for i in np.arange(len(list_of_cat)):
    #Gets each category list
    #
    response = category_addition(list(all_strats), list_of_cat[i])
    total_words = np.append(total_words, response)
    


In [156]:
len(category_addition_correct(list(all_strats), hobbies))

107

In [137]:
Social_Connection_Relationships = family + friends + connecting + virtual + pets

set(Social_Connection_Relationships)

aux = pd.Series(list(category_addition_correct(list(all_strats), food_and_drink)))
                
df1 = aux.to_frame()
df2 = aux.value_counts().to_frame()

aux2 = pd.Series(list(category_addition_correct(list(all_strats), value_time)))
                
df3 = aux2.to_frame()
df4 = aux2.value_counts().to_frame()

aux3 = pd.Series(list(category_addition_correct(list(all_strats), chores_and_projects)))
                
df5 = aux3.to_frame()
df6 = aux3.value_counts().to_frame()

aux4 =  pd.Series(list(category_addition_correct(list(all_strats), humor)))
                
df7 = aux4.to_frame()
df8 = aux4.value_counts().to_frame()

aux5 =  pd.Series(list(category_addition_correct(list(all_strats), weather)))
                
df9 = aux5.to_frame()
df10 = aux5.value_counts().to_frame()

aux6 = pd.Series(list(category_addition_correct(list(all_strats), music)))
                
df11 = aux6.to_frame()
df12 = aux6.value_counts().to_frame()

aux7 =  pd.Series(list(category_addition_correct(list(all_strats), hobbies)))
                
df13 = aux7.to_frame()
df14 = aux7.value_counts().to_frame()






In [138]:
writer = pd.ExcelWriter('hoobies.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
df13.to_excel(writer, sheet_name='hobby1')
df14.to_excel(writer, sheet_name='hobby2')



# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [15]:
#len(category_addition(aux2strat, religion))
#239
len(category_addition(list(all_strats), religion))

632

In [18]:
needles = list(set(total_words))

y = category_addition(list(all_strats), needles)
y

KeyboardInterrupt: 

In [113]:
#uncategorized = []
#for i in list(set(all_strats)):
#    if i not in list(set(y)):  
#        uncategorized = np.append(uncategorized, i)

unique_all_strats = np.unique(all_strats)
unique_y = np.unique(y)

uncategorized = [x for x in unique_all_strats if x not in unique_y]
len(uncategorized)

1889

In [114]:
list(uncategorized)

['',
 ' blood donation',
 ' flowering trees blooming',
 ' solidarity',
 '"doing my part"',
 '"feel good" stories',
 '"we are all in this together!"',
 'a bath',
 'a belief that good can come out of these experiences.',
 'a chance to get some things done at home',
 'a day off',
 'a good reflection on what is important.',
 'a lot of reflection on what we take for granted',
 'a more relaxed atmosphere throughout the day',
 'a new significant other',
 'a plan to meet needs',
 'a proper diagnosis',
 'a slowed down pace of life',
 'a sunny day',
 'a surprise delivery',
 'ability to provide necessities',
 'able to get groceries!!',
 'able to reset my mind',
 'access to daily comforts',
 'access to resources',
 'accessible free online professional development opportunities',
 'accomplishing new things',
 'accomplishment',
 'accomplishments',
 'achieving goals',
 'actions to bring back normalacy',
 'activity',
 'actually feeling less stress',
 'adaptability',
 'adjustment to distance learning',

## The number of responses in eache category

In [14]:
print("number of people citing family as cause of joy: ",len(category_addition(list(all_strats), family)))
#5481/len(list(all_strats))
print()
print("number of people citing religion as cause of joy: ", len(category_addition(list(all_strats), religion)))
#632
print()
print("number of people citing athletics as cause of joy: ",len(category_addition(list(all_strats), athletics)))
#1343
print()
print("number of people citing outdoors as cause of joy: ",len(category_addition(list(all_strats), outdoors)))
#794
print()
print("number of people citing pets as cause of joy: ",len(category_addition(list(all_strats), pets)))
#551
print()
print("number of people citing virtual connection as cause of joy: ",len(category_addition(list(all_strats), virtual)))
#997
print()
print("number of people citing friends as cause of joy: ", len(category_addition(list(all_strats), friends)))
#1638
print()
print("number of people citing valuing new time as cause of joy: ",len(category_addition(list(all_strats), value_time)))
#390
print()
print("number of people citing food and drink as cause of joy: ",len(category_addition(list(all_strats), food_and_drink))) 
#408
print()
print("number of people citing chores and projects as cause of joy: ",len(category_addition(list(all_strats), chores_and_projects))) 
#126
print()
print("number of people citing compassion,kindness, gratitude, kindness as cause of joy: ",len(category_addition(list(all_strats), compassion)))
#312

print("number of people citing music as cause of joy: ",len(category_addition(list(all_strats), music)))
#148
print()
print("number of people citing chores_and_projects as cause of joy: ",len(category_addition(list(all_strats), chores_and_projects)))
#126
print()
print("number of people citing weather as cause of joy: ",len(category_addition(list(all_strats), weather)))
#162
print()
print("number of people citing humor as cause of joy: ",len(category_addition(list(all_strats), humor)))
#215
print()
print("number of people citing engage_with_work_and_school as cause of joy: ",len(category_addition(list(all_strats), engage_with_work_and_school)))
#971
print()
print("number of people citing connecting as cause of joy: ", len(category_addition(list(all_strats), connecting)))
# #1264
print()
print("number of people citing hobbies as cause of joy: ",len(category_addition(list(all_strats), hobbies)))
# #108
print()
print("number of people citing helping as cause of joy: ", len(category_addition(list(all_strats), helping)))
# #259


971/len(list(all_strats))

number of people citing family as cause of joy:  5481

number of people citing religion as cause of joy:  632

number of people citing athletics as cause of joy:  1343

number of people citing outdoors as cause of joy:  794

number of people citing pets as cause of joy:  551

number of people citing virtual connection as cause of joy:  997

number of people citing friends as cause of joy:  1707

number of people citing valuing new time as cause of joy:  390

number of people citing food and drink as cause of joy:  408

number of people citing chores and projects as cause of joy:  126

number of people citing compassion,kindness, gratitude, kindness as cause of joy:  312
number of people citing music as cause of joy:  148

number of people citing chores_and_projects as cause of joy:  126

number of people citing weather as cause of joy:  162

number of people citing humor as cause of joy:  215

number of people citing engage_with_work_and_school as cause of joy:  971

number of people c

0.06271394432603501

In [99]:
# uncategorized = []

# list(set(all_strats)) not in list(set(y))

In [54]:
list(uncategorized)


[]

In [21]:
all_self_strategies = []
aux = np.append(all_self_strategies, df['WellBeingSupport_16'])
aux1 = np.append(aux, df['WellBeingSupport_17'])
aux2strat = np.append(aux1, df['WellBeingSupport_18'])

all_strats = pd.Series(pd.Series([x.lower() for x in aux2strat]))
ordered_counts_strat = all_strats.value_counts()

# Most Used Strategies & Number of Unique Strategies

# Look at the resources that people need

In [22]:
print(ordered_counts_strat)
print("Number of Unique Strategies before cleaning:", len(ordered_counts_strat))

                                                                  3309
communication                                                       50
not sure                                                            32
?                                                                   27
resources                                                           25
                                                                  ... 
connecting with them                                                 1
daily messages to teachers for their well being                      1
provide some type of structure and continuity                        1
allocating resources to families. such as ipads and or laptops       1
we are all in this together, fortunately, and unfortunately!         1
Length: 11531, dtype: int64
Number of Unique Strategies before cleaning: 11531


In [19]:
exercise = ['walk', "tai chi", "water", "stretch", "jog", "zumba", "dance", "yoga", "isometrics", "volleyball",\
            'skat', 'ski', 'wrest', 'baseball', 'calisthenics', 'jump', 'bike', 'skateboard', 'surf', 'climb',\
            'tennis', 'canoe','kayak', 'row', 'hockey', 'soccer', 'diving','jump', 'sprint', 'throw', 'basketball',\
            'hurdle', 'football', 'lift','crossfit', 'boxing', 'triathlon', 'ballet', 'gymn', 'squats']
new_all_strat = []

for i in list(all_strats):
    if any(word in i for word in exercise):
        new_all_strat =np.append(new_all_strat, 'exercise')
    else:
        new_all_strat =np.append(new_all_strat, i)
    

KeyboardInterrupt: 

In [ ]:
new_all_strats = new_all_strat)
ordered_counts_strat = new_all_strats.value_counts()


# Causes of stress and joy

-Bucket
-
-
-
-


Categories joy
exercise outdoors faith play 

We are looking at data from teachers and business people.

Emotion regulation
    People respond with a list of strategie
    
    
Looking at things that effect emotions